<a href="https://colab.research.google.com/github/Nazhan99/Tensorflow_notes/blob/main/04_Transfer_learning_in_tensorflow_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Transfer learning with tensorflwo part 1: Feature Extraction

Transfer learning is leveraging a working model's existing architecture and learned pattern for our own data.

The two main benefits:
1. Can leverage an existing neural network architecture proven to work on problems similar to our own.
2. Can leverage a working neural network architecture which has already learned patterns on similar data to our own, then we can adapt those patterns to our own data,

In [1]:
#Are we using a GPU?
!nvidia-smi

Fri Feb 18 07:23:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Download and becoming one with the data

In [3]:
#Get data (10% of 10 food classes from Food101)
import zipfile

#download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

#unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

--2022-02-18 07:26:17--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.197.128, 64.233.191.128, 209.85.145.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   201MB/s    in 0.8s    

2022-02-18 07:26:18 (201 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [5]:
#how many images in each folder
import os

#walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'. ")


There are 2 directories and 0 images in '10_food_classes_10_percent'. 
There are 10 directories and 0 images in '10_food_classes_10_percent/test'. 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'. 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'. 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'. 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'. 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'. 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'. 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'. 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'. 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'. 
There are 0 directories and 250 images in '10_food_classes_10_perce